In [100]:
import pandas as pd
import numpy as np
from functions import *
from pytorch_util import *
from pytorch_models import ConvBatchLeaky1D,GRU_NCL,LambdaLayer,biasLayer,\
                        MultiHeadedAttention,PositionwiseFeedForward,SublayerConnection
from torch.nn import Linear,Sequential,LeakyReLU,Dropout,L1Loss
from torch.utils.data import DataLoader
from torch.optim import Adam,lr_scheduler

In [28]:
## change ##
batch_size = 16
clip = .1
block = ResidualBlockGLU
normalFun = normalize_log2
#normalFun = normalize_log
name = 'Spec_2d_cnn_shareWeight_attention'

In [3]:
train = pd.read_csv("../Data/train.csv", 
                    dtype={'acoustic_data': np.float32, 'time_to_failure': np.float32})
train = train.values

Prepare data

In [4]:
train_gen = SequenceGenSpec(train,normalFun=normalFun)
#train_gen = mixupWrapper(SequenceGenSpec(train),4e-2) # does not improve
train_gen = DataLoader(train_gen,batch_size,False,num_workers=2)

val_gen = SequenceGenSpec(train,False,normalFun=normalFun)
val_gen = DataLoader(val_gen,batch_size,False,num_workers=2)

In [5]:
x,y = next(iter(train_gen))

In [6]:
x.shape

torch.Size([16, 129, 780])

In [18]:
model(x).shape

torch.Size([16])

In [11]:
258/2/3

43.0

Build model

In [91]:
class EncoderLayer(nn.Module):
    "Encoder is made up of self-attn and feed forward (defined below)"
    def __init__(self, size, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = MultiHeadedAttention(2,size,dropout)
        self.feed_forward = PositionwiseFeedForward(size,size,dropout)
        self.sublayer = nn.ModuleList([SublayerConnection(size, dropout),SublayerConnection(size, dropout)])
        self.size = size

    def forward(self, x, mask=None):
        "Follow Figure 1 (left) for connections."
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
        return self.sublayer[1](x, self.feed_forward)   

In [136]:
model = Sequential(Dropout(0.5),
                   ConvBatchLeaky1D(129,256,5,stride=2),
                   LambdaLayer(lambda x:x.transpose(1,2)),
                   EncoderLayer(256,0.5),
                   Linear(256,1,bias=False),
                   Dropout(0.2),
                   LambdaLayer(lambda x:x.squeeze(2).mean(1,keepdim=True)),
                   biasLayer((1,)),
                   LambdaLayer(lambda x:x.squeeze(1))
                      ).to('cuda:0')

In [137]:
opt = Adam(trainable_parameter(model),lr=2e-4)

In [138]:
loss_func_nor = loss_func_generator(L1Loss())

Training

In [139]:
model = fit(5, model, loss_func_nor, opt, train_gen, val_gen,clip=clip)

epoch:0, train_loss:2.3939479486300397, val_loss:2.4598631858825684
epoch:1, train_loss:2.1571697506767054, val_loss:2.4444499015808105
epoch:2, train_loss:2.1166211028511706, val_loss:2.5073161125183105
epoch:3, train_loss:2.124555957431977, val_loss:2.4908926486968994
epoch:4, train_loss:2.0987318232655525, val_loss:2.495560646057129
Training completed in 27.856075286865234s


Submission

In [141]:
make_submission(name,model,normalFun,batch_size)